In [1]:
import h5py
import numpy as np

In [2]:
# Load the data
train_dataset = h5py.File('Te.h5', "r")
test_dataset = h5py.File('Tr.h5', "r")

In [3]:
# Print the keys in the datasets
print("Keys in train_dataset:", list(train_dataset.keys()))
print("Keys in test_dataset:", list(test_dataset.keys()))

Keys in train_dataset: ['images', 'labels']
Keys in test_dataset: ['images', 'labels']


In [4]:
# Based on the output, use the correct keys to access the data
# For example, if the keys are 'X' and 'Y':
train_x = np.array(train_dataset["images"][:])  # train set features
train_y = np.array(train_dataset["labels"][:])  # train set labels
test_x = np.array(test_dataset["images"][:])  # test set features
test_y = np.array(test_dataset["labels"][:])  # test set labels

In [5]:
# Flatten the images to be compatible with the softmax regression model
train_x_flatten = train_x.reshape(train_x.shape[0], -1).T
test_x_flatten = test_x.reshape(test_x.shape[0], -1).T

In [6]:
# Standardize the data
train_x = train_x_flatten / 255.
test_x = test_x_flatten / 255.


In [7]:
# Define functions for softmax regression model
def initialize_parameters(n_x, n_y):
    W = np.zeros((n_y, n_x))
    b = np.zeros((n_y, 1))
    return W, b

def forward_propagation(X, W, b):
    Z = np.dot(W, X) + b
    A = np.exp(Z) / np.sum(np.exp(Z), axis=0, keepdims=True)
    return A

def compute_cost(A, Y):
    m = Y.shape[1]
    cost = -1/m * np.sum(Y * np.log(A + 1e-8))
    return cost

def backward_propagation(X, A, Y):
    m = Y.shape[1]
    dZ = A - Y
    dW = 1/m * np.dot(dZ, X.T)
    db = 1/m * np.sum(dZ, axis=1, keepdims=True)
    return dW, db

def gradient_descent(X, Y, learning_rate, num_iterations):
    n_x, m = X.shape
    n_y = Y.shape[0]
    
    W, b = initialize_parameters(n_x, n_y)
    costs = []
    
    for i in range(num_iterations):
        A = forward_propagation(X, W, b)
        cost = compute_cost(A, Y)
        dW, db = backward_propagation(X, A, Y)
        
        W -= learning_rate * dW
        b -= learning_rate * db
        
        if i % 100 == 0:
            costs.append(cost)
            print(f"Cost after iteration {i}: {cost}")
    
    return W, b, costs

def predict(X, W, b):
    A = forward_propagation(X, W, b)
    return np.argmax(A, axis=0)


In [8]:
print(f"train_x shape: {train_x.shape}")
print(f"train_y shape: {train_y.shape}")
print(f"test_x shape: {test_x.shape}")
print(f"test_y shape: {test_y.shape}")

train_x shape: (49152, 50)
train_y shape: (50,)
test_x shape: (49152, 250)
test_y shape: (250,)


In [9]:
# Ensure train_y and test_y are in the correct format
train_y = train_y.astype(int)  # Ensure they are integer types
test_y = test_y.astype(int)


In [10]:
# One-hot encode the labels if needed
num_classes = np.max(train_y) + 1
train_y = np.eye(num_classes)[train_y.reshape(-1)].T
test_y = np.eye(num_classes)[test_y.reshape(-1)].T

In [11]:
# Train the model
learning_rate = 0.01
num_iterations = 2000
W, b, costs = gradient_descent(train_x, train_y, learning_rate, num_iterations)

Cost after iteration 0: 1.6094378624341017
Cost after iteration 100: 0.005749674368794678
Cost after iteration 200: 0.00322479175431572
Cost after iteration 300: 0.0023883301595754407
Cost after iteration 400: 0.001942488395304682
Cost after iteration 500: 0.001655465463129339
Cost after iteration 600: 0.0014513864507450494
Cost after iteration 700: 0.0012971534038784208
Cost after iteration 800: 0.0011756714490354985
Cost after iteration 900: 0.0010770753895362152
Cost after iteration 1000: 0.000995208078554716
Cost after iteration 1100: 0.0009259961483265897
Cost after iteration 1200: 0.0008666208974088211
Cost after iteration 1300: 0.0008150609549139252
Cost after iteration 1400: 0.0007698240711556926
Cost after iteration 1500: 0.0007297818115038818
Cost after iteration 1600: 0.000694063402477764
Cost after iteration 1700: 0.0006619851725545525
Cost after iteration 1800: 0.000633002256613806
Cost after iteration 1900: 0.0006066746943327164


In [12]:
# Make predictions on the test set
predictions = predict(test_x, W, b)
test_y_labels = np.argmax(test_y, axis=0)

In [13]:
# Calculate accuracy
accuracy = np.mean(predictions == test_y_labels)
print(f"Test accuracy: {accuracy * 100:.2f}%")

Test accuracy: 36.00%


In [14]:
import random
import numpy as np

# Select a random image from the test set
random_index = random.randint(0, test_x.shape[1] - 1)
random_image = test_x[:, random_index].reshape(-1, 1)

# Make a prediction on the random image
predicted_label = predict(random_image, W, b)

# Get the actual label
actual_label = np.argmax(test_y[:, random_index])

# Display the predicted and actual labels
print(f"Predicted label: {predicted_label[0]}")
print(f"Actual label: {actual_label}")

Predicted label: 0
Actual label: 1
